In [15]:
import pandas as pd

In [16]:
data = pd.read_csv("dataset.csv", delimiter=';')

In [17]:
train_data = []

In [18]:
for _, row in data.iterrows():
    text = row['text']
    origin = row['origin']
    destination = row['destination']
    detours = row['detours'] if pd.notna(row['detours']) else ""
    
    # Vérifier que le texte est en français et que c'est un itinéraire valide
    if origin != "NOT_FRENCH" and origin != "NOT_TRIP":
        # Trouver les positions d'origine et de destination dans le texte
        start_origin = text.lower().find(origin)
        end_origin = start_origin + len(origin)
        start_destination = text.lower().find(destination)
        end_destination = start_destination + len(destination)
        
        # Liste temporaire pour les positions uniques de détour
        detour_positions = []
        
        if detours:
            for detour in detours.split(","):
                detour = detour.strip()
                start_detour = text.lower().find(detour)
                end_detour = start_detour + len(detour)
                
                # Vérifier que le détour ne chevauche pas ORIGIN ou DESTINATION
                if start_detour >= 0 and (
                    end_detour <= start_origin or start_detour >= end_origin
                ) and (
                    end_detour <= start_destination or start_detour >= end_destination
                ):
                    # Vérifier qu'il n'y a pas de chevauchement avec d'autres détours déjà ajoutés
                    overlap = any(
                        (start < end_detour and end > start_detour)
                        for start, end, _ in detour_positions
                    )
                    if not overlap:
                        detour_positions.append((start_detour, end_detour, "DETOUR"))
        
        # Ajouter les annotations d'origine, destination et détours sans chevauchements
        if start_origin >= 0 and start_destination >= 0 and end_origin <= start_destination:
            entities = [
                (start_origin, end_origin, "ORIGIN"),
                (start_destination, end_destination, "DESTINATION")
            ] + detour_positions  # Ajouter les détours non-chevauchants

            train_data.append((text, {"entities": entities}))

In [19]:
import spacy
from spacy.training.example import Example
import random

In [20]:
nlp = spacy.blank("fr")

In [21]:
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

In [22]:
ner.add_label("ORIGIN")
ner.add_label("DESTINATION")
ner.add_label("DETOUR")

1

In [23]:
optimizer = nlp.begin_training()

In [24]:
for itn in range(20):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], losses=losses, drop=0.5, sgd=optimizer)
    print(f"Itération {itn} - Losses: {losses}")


/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "en passant par bessay sur allier coubert calavante..." with entities "[(55, 59, 'ORIGIN'), (96, 102, 'DESTINATION'), (15...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "en passant par beaumont sur vesle et montigny en c..." with entities "[(77, 80, 'ORIGIN'), (90, 106, 'DESTINATION'), (15...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.

Itération 0 - Losses: {'ner': 11584.51143947512}
Itération 1 - Losses: {'ner': 5326.365459161355}
Itération 2 - Losses: {'ner': 4265.298669417285}
Itération 3 - Losses: {'ner': 3883.2248929012803}
Itération 4 - Losses: {'ner': 3370.4420408958463}
Itération 5 - Losses: {'ner': 3073.289576310449}
Itération 6 - Losses: {'ner': 2842.5799983024935}
Itération 7 - Losses: {'ner': 2705.258543145459}
Itération 8 - Losses: {'ner': 2602.3461913991005}
Itération 9 - Losses: {'ner': 2441.866069677474}
Itération 10 - Losses: {'ner': 2360.8109670044996}
Itération 11 - Losses: {'ner': 2288.90263013482}
Itération 12 - Losses: {'ner': 2183.6871832169886}
Itération 13 - Losses: {'ner': 2142.743837976162}
Itération 14 - Losses: {'ner': 2067.8844725613253}
Itération 15 - Losses: {'ner': 1996.9890803255498}
Itération 16 - Losses: {'ner': 1923.0550091124899}
Itération 17 - Losses: {'ner': 1755.896679619108}
Itération 18 - Losses: {'ner': 1800.218348164056}
Itération 19 - Losses: {'ner': 1790.5218247681667}


In [ ]:
# range essayé : 10, 20

In [41]:
nlp.to_disk("model_test")

In [ ]:
nlp = spacy.load('./model_test')

In [6]:
from langdetect import detect, LangDetectException

In [7]:
def is_french(text):
    try:
        return detect(text) == "fr"
    except LangDetectException:
        return False

In [25]:
from unidecode import unidecode
import re

In [30]:
def test_trip_request(text):
    if not is_french(text):
        return "NOT_FRENCH"
    
    text = unidecode(text).lower()
    text = re.sub(r'[^\w\s]', '', text)

    doc = nlp(text)
    origin, destination = None, None
    detours = []

    for ent in doc.ents:
        if ent.label_ == "ORIGIN":
            origin = ent.text
        elif ent.label_ == "DESTINATION":
            destination = ent.text
        elif ent.label_ == "DETOUR":
            detours.append(ent.text)
    
    if origin and destination:
        return (text, origin, destination, detours)
    else:
        return "NOT_TRIP"

In [9]:
def show_infos(origin, destination, detours):
    print(f'Depart: {origin}')
    print(f'Arrivée: {destination}')
    detours_sentence = ""
    for i in range(len(detours)):
        if(i == len(detours) - 1):
            detours_sentence += detours[i]
        else:
            detours_sentence += detours[i] + ", "
    print(f'Détours: {detours_sentence if len(detours) > 0 else "Aucun"}\n')

In [36]:
test_phrases = [
    "Je veux aller de paris à lyon",
    "J'aimerai aller de lille à nice",
    "Voyage de rouen jusqu'à nice",
    "Quel est le trajet de toulouse à bordeaux ?",
    "Je veux aller de paris à lyon en passant par nice",
    "En passant par toulouse, je veux aller de paris à lyon",
    "J'aime bien les restaurants de paris",
    "What time is it in Paris ?",
    "Quel est le trajet de strasbourg à bordeaux ?",
    "Quel est le trajet de bordeaux à strasbourg en passant par lyon ?",
    "Comment me rendre à strasbourg depuis nice ?",
    "En passant par Lyon, j'aimerai aller à Nice depuis Strasbourg",
]

In [37]:
for phrase in test_phrases:
    print(f"sentence: {phrase}")
    print(test_trip_request(phrase.lower()))

sentence: Je veux aller de paris à lyon
('je veux aller de paris a lyon', 'paris', 'lyon', [])
sentence: J'aimerai aller de lille à nice
('jaimerai aller de lille a nice', 'lille', 'nice', [])
sentence: Voyage de rouen jusqu'à nice
('voyage de rouen jusqua nice', 'rouen', 'nice', [])
sentence: Quel est le trajet de toulouse à bordeaux ?
('quel est le trajet de toulouse a bordeaux ', 'toulouse', 'bordeaux', [])
sentence: Je veux aller de paris à lyon en passant par nice
('je veux aller de paris a lyon en passant par nice', 'paris', 'lyon', ['nice'])
sentence: En passant par toulouse, je veux aller de paris à lyon
('en passant par toulouse je veux aller de paris a lyon', 'paris', 'lyon', ['toulouse'])
sentence: J'aime bien les restaurants de paris
NOT_TRIP
sentence: What time is it in Paris ?
NOT_FRENCH
sentence: Quel est le trajet de strasbourg à bordeaux ?
('quel est le trajet de strasbourg a bordeaux ', 'strasbourg', 'bordeaux', [])
sentence: Quel est le trajet de bordeaux à strasbour

In [40]:
phrase = "En passant par Lyon, je veux aller de Nice à Strasbourg"
phrase = unidecode(phrase).lower()
phrase = re.sub(r'[^\w\s]', '', phrase)
test_trip_request(phrase)

('en passant par lyon je veux aller de nice a strasbourg',
 'nice',
 'strasbourg',
 ['lyon'])